In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install -y fonts-nanum*

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from collections import deque
import requests
import random
import logging

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)

webdriver = webdriver.Chrome('chromedriver', chrome_options = chrome_options)

<ipython-input-2-475e698df5fe>:7: DeprecationWarning: use options instead of chrome_options
  webdriver = webdriver.Chrome('chromedriver', chrome_options = chrome_options)


# 카테고리 별로 url 가져오기

#### - url

In [17]:
url = 'https://www.wadiz.kr/web/wreward/category/312?keyword=&endYn=ALL&order=recommend'
webdriver.get(url)

In [18]:
last_height = webdriver.execute_script("return document.body.scrollHeight")
body = webdriver.find_element_by_tag_name("body")
while True:
    btn = webdriver.find_element_by_css_selector('#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div:nth-child(2) > div > button')
    if btn:
        try:

            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

        except:
            print('버튼 안됨/타임오류')
            break
        
    else:
        print('버튼 없음')
        break
        

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div:nth-child(2) > div > button"}
  (Session info: chrome=89.0.4389.128)


##### 하다가 안되면 멈추면 그냥 interrupt하세요

#### - atag 가져오기

In [19]:
try:
    atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
    #list_href = [a.get_attribute('href') for a in atag]
except:
    print('태그 업서요')

#### -리스트로 저장

In [20]:
list_href = [a.get_attribute('href') for a in atag]

#### -데이터프레임으로 

In [21]:
data = {'link' : list_href}
link_df = pd.DataFrame(data)
link_df.drop_duplicates(inplace=True)

In [22]:
link_df

,link
0,https://www.wadiz.kr/web/campaign/detail/794
2,https://www.wadiz.kr/web/campaign/detail/22371
4,https://www.wadiz.kr/web/campaign/detail/5024
6,https://www.wadiz.kr/web/campaign/detail/692
8,https://www.wadiz.kr/web/campaign/detail/25095
10,https://www.wadiz.kr/web/campaign/detail/27351
12,https://www.wadiz.kr/web/campaign/detail/1042
14,https://www.wadiz.kr/web/campaign/detail/89184
16,https://www.wadiz.kr/web/campaign/detail/23797
18,https://www.wadiz.kr/web/campaign/detail/29605


#### -csv로저장

In [23]:
link_df.to_csv('donation_link_df')

# 특성 가져오기

#### - 데이터프레임 가져오기

In [4]:
link_df = pd.read_csv('tech_link_df')

#### -리스트로 형변환

In [23]:
link_list = list(link_df['link'].values)

In [24]:
len(link_list)

2126

#### -특성가져오기

In [ ]:
titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
n_supporters, likes = [],[]
durations, target_amounts = [],[]
descriptions = []
visited = []
error

while link_list:
    try:       
        #리스트에서 맨 앞에 요소 빼오기
        url = link_list.pop(0)
        #방문한 링크 리스트 저장
        visited.append(url)
        webdriver.get(url)
        soup = bs(webdriver.page_source, 'html.parser')
        time.sleep(0.3*random.randint(2,4))

        #titles, categories
        category = soup.find('p',class_ = 'title-info').text
        categories.append(category)
        title = soup.find('h2',class_ = 'title').text
        titles.append(title)

        #days_left, percentages, funding_amounts, n_supporters
        info = soup.select_one('#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.wd-ui-campaign-content > div > div.project-state-info > div.state-box').text.split('\n')
        day = info[1]
        days_left.append(day)
        percentage = info[3].split(' ')[0]
        percentages.append(percentage)
        funding = info[4].split(' ')[0]
        funding_amounts.append(funding)
        supporter = info[5].split(' ')[0].split('명')[0]
        n_supporters.append(supporter)

        #company_names 
        company_name = soup.find("div", {"id":"reward-report-content2"})['data-maker-name']
        company_names.append(company_name)

        #likes
        like = soup.select_one('#cntLike').text
        likes.append(like)

        #durations,target_amounts
        info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
        duration = info[2].split(' ')[-1]
        durations.append(duration)
        target_amount = info[1].split(' ')[-1]
        target_amounts.append(target_amount)

        #descriptions
        description = soup.select_one("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")
        descriptions.append(description.text.split('\n')[1].strip())
 
    except:
        continue
   

In [19]:
len(visited)

76

In [227]:
#계속 76개만 가져옴 (timesleep가져온 후에도)

#### -데이터 프레임 만들기

In [21]:
data = {
    'categories':categories,
    'title':titles,
    "company_names":company_names,
    'percentages':percentages, 
    'funding_amounts' :funding_amounts, 
    'days_left' :days_left
} 

df = pd.DataFrame(data)
df['n_supporters'] = n_supporters
df['likes'] = likes
df['descriptions'] = descriptions
df['durations'] = durations
df['target_amounts'] = target_amounts


In [22]:
df

,categories,title,company_names,percentages,funding_amounts,days_left,n_supporters,likes,descriptions,durations,target_amounts
0,테크·가전,[앵콜]신발냄새 잡는 끝판왕! 신발관리의 혁명 홈세라신발청정기U,주식회사 홈세라,846%,"8,461,600원",11일 남음,238,182,[가성비 끝판왕 1만원대] 신발 관리의 혁명! 신발도 청정하는 시대! 홈세라 신발살...,2021.04.14-2021.05.10,"1,000,000원"
1,테크·가전,손가락 하나로 디지털 기기를 컨트롤! 진짜 만능 블루투스 마우스&리모컨,주식회사 블루티움,4653%,"23,268,500원",25일 남음,404,99,"전자책, 웹툰, 인강, 마우스, 영상촬영이 하나의 리모컨으로 다 된다고? 더욱 스...",2021.04.28-2021.05.24,"500,000원"
2,테크·가전,핑구X듀얼충전 마우스패드를 2만원대로! [핑구's Back2],케이티씨에스,1026%,"10,268,000원",18일 남음,291,68,전자기기 2개쯤은 기본이잖아요. 1+1 동시충전과 더불어 마우스 패드가 만났습니다....,2021.04.28-2021.05.17,"1,000,000원"
3,테크·가전,[얼룩제거히어로!2차앵콜]언제나!물만있으면OK무선초음파세척스틱!웨이블린!,주식회사 엘파인,3459%,"34,592,000원",18일 남음,604,285,"1주일 적어도 30회이상 얼룩빌런에 노출되는 일상-일터,집,식사시간,저녁약속,커피타...",2021.04.27-2021.05.17,"1,000,000원"
4,테크·가전,[유인+흡입+감전] 자비없는 3콤보 해충사냥꾼 듀얼 해충퇴치기 등장!,동양라인,6715%,"134,318,100원",15일 남음,"2,638",1034,올 여름 필수템! 모기에게 물리지 않는 법은 물리기 전에 잡는 겁니다!,2021.04.23-2021.05.14,"2,000,000원"
...,...,...,...,...,...,...,...,...,...,...,...
70,테크·가전,[앵콜]여행용기타-모가비(녹음+믹싱+BT스피커+폴더블),주식회사 짐,30477%,"152,388,000원",펀딩성공,253,316,시끄러운 장소에서 잡음없이 깨끗한 연주원음을 녹음 할수 있습니다. 스마트폰과 연결하...,2021.03.26-2021.04.26,"500,000원"
71,테크·가전,착! 붙이는 보조배터리 #무선 #고속 #말도안되는초강력스펙까지!,착한텔레콤 커넥팅팟,30174%,"150,870,343원",펀딩성공,"4,481",2728,"진~짜 들고다니기 편한 무선 보조배터리! 파워풀한 스펙부터, 쓸모 있는 다양한 기능...",2018.09.18-2018.10.31,"500,000원"
72,테크·가전,[글로벌8억펀딩] 번역기 아마존 베스트셀러 타임캐틀! 자유롭게 대화하세요,브이제이센타,29172%,"145,860,950원",펀딩성공,"1,115",1493,미국 펀딩 사이트 인디고고에서 75만 달러(한화 8억) 높은 펀딩금액으로 성공한 올...,2020.11.17-2020.12.14,"500,000원"
73,테크·가전,의사를 포기하고 5년간 눈만 연구해서 만든 [눈 근육 운동기],주식회사에덴룩스,29059%,"435,895,010원",펀딩성공,"1,227",1725,"오투스플러스는 게임 APP과 연동하여 눈 속 근육을 꾸준히 운동할 수 있으며, 개인...",2020.10.30-2020.11.23,"1,500,000원"


In [ ]:
df.to_csv('#카테고리 이름으로 바꿔주세요')

# 이거는 무시해 주세요

In [ ]:
def crawling(category):
    
    url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnJ4L-U5ZdU8R_6y2WpcNpXM6WVwglke6OpUP-fekXUudZSBTXWrlRAaAg0xEALw_wcB'
    webdriver.get(url)
    
    categories_sel = webdriver.find_elements_by_css_selector('#contents > section.reward-category > div > ul > div > div > li.category-slide.slick-slide.slick-current.slick-active > a')
    page_source = webdriver.page_source
    #카테고리명 가져오기
    soup = bs(webdriver.page_source, 'html.parser')
    cate = soup.findAll('span', class_ = 'desc-category')
    categories = [cat.text for cat in cate]
    
    #
    search = webdriver.find_element_by_css_selector(f'#contents > section.reward-category > div > ul > div > div > li:nth-child({categories.index(category)+1}) > a')
    search.click()

    #스크롤 다운
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body = webdriver.find_element_by_tag_name("body")
    while True:
        last_height = webdriver.execute_script("document.body.scrollHeight")
        scroll_down = 0
        while scroll_down < 10:

            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

            time.sleep(0.3)

            scroll_down += 1

        new_height = webdriver.execute_script("document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
    
    
    #titles, categories, company_names, percentages, funding_amounts, days_left
    titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
    contents = webdriver.find_elements_by_class_name('ProjectCardList_item__1owJa')
    for content in contents:
        split = content.text.split('\n')
        if len(split) > 5:
            cont = deque(split)
            cont.popleft()
            title, category, company_name, target, day_left = list(cont)
        else:
            title, category, company_name, target, day_left = content.text.split('\n')

        titles.append(title)
        categories.append(category)
        company_names.append(company_name)
        percentages.append(target.split('%')[0])
        funding_amounts.append(target.split('%')[1])
        days_left.append(day_left)

    data = {
    'title':titles,
    'categories':categories,
    "company_names":company_names,
    'percentages':percentages, 
    'funding_amounts' :funding_amounts, 
    'days_left' :days_left
    } 

    
    
    n_supporters, likes = [],[]
    durations, target_amounts = [],[]
    descriptions = []
    
    #테그 가져오기
    atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
    list_href = [a.get_attribute('href') for a in atag]
    
#     return dataframe, list_href

    for href in list_href:
        url = href
        webdriver.get(url)
        time.sleep(0.5)
        page_source = webdriver.page_source
        soup = bs(page_source, 'html.parser')

        #n_supporters, likes
        supporter = soup.find('p',class_ = 'total-supporter').text.split('명')[0]
        n_supporters.append(supporter)
        like = soup.find('em',class_='cnt-like').text
        likes.append(like.strip())

        #durations,target_amounts
        info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
        duration = info[2].split(' ')[-1]
        durations.append(duration)
        target_amount = info[1].split(' ')[-1]
        target_amounts.append(target_amount)

        #descriptions
        description = soup.select("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")[0]
        descriptions.append(description.text.strip())
        
 
  
    #데이터 프레임 생성
 
    df = pd.DataFrame(data)
    df['percentages'] = percentages
    df['funding_amounts'] = funding_amounts
    df['n_supporters'] = n_supporters
    df['likes'] = likes
    df['descriptions'] = descriptions
    df['durations'] = durations
    df['target_amounts'] = target_amounts

        
    return df.to_csv(category+'_df')


In [ ]:
def timeseries_data(category):
        
    #하나씩 클릭해서 들어가야함 
    page_source = webdriver.page_source
    soup = bs(page_source, 'html.parser')
    contents = soup.findAll('div',class_='ProjectCardList_item__1owJa')
    
    for i in range(1,len(titles)+1): #len(contents)
        btn = webdriver.find_element_by_css_selector(f'div.ProjectCardList_container__3Y14k > div > div:nth-child({i}) > div > div > a > div > span')
        time.sleep(0.5)
        btn.click()
        
        #시계열 데이터
        support_page = webdriver.find_element_by_css_selector('#container > div.reward-nav > ul > li:nth-child(6)')
        time.sleep(1)
        support_page.click()

        button = webdriver.find_element_by_css_selector('#reward-static-supports-list-app > div > div > div > div.ListMoreButton_container__1rOIU > button')
        while button:
            try:
                button.click()
                time.sleep(2)
            except:
                break

        supports = []
        page_source = webdriver.page_source
        soup = bs(page_source, 'html.parser')
        supports_info = soup.findAll('div', class_ = 'RewardSupporterItem_container__1UTDZ')

        for i in supports_info:
            supports.append(i.text.split('\xa0'))
        #     데이터 전처리 필요함

        webdriver.back()
        time.sleep(0.5)
        webdriver.back()

    
    data = supports
    df = pd.DataFrame(data)
    
    return df.to_csv(category+'_tiemseires_df')

    
